In [64]:
%reset
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import scipy.stats
import mne
from mne.time_frequency import tfr_morlet
from mne.stats import permutation_cluster_1samp_test
import gc
import os
import copy
import pickle
from os.path import exists
import mne
import numpy as np
from mne import create_info
from IPython.utils import io
import yasa
import seaborn as sns

gc.collect()

11

In [65]:
pkl_dir = "C:\\Users\\User\\Cloud-Drive\\BigFiles\\OmissionExpOutput\\\imported_eventDetectionChan"
import_type = "eventDetectionChan"
output_dir_name = 'eventDetection\\done_on_imported'

In [66]:
import_path = f'{pkl_dir}\\{import_type}.pkl'

with open(import_path, "rb") as file:
    [allsubsdata_perFile, configu] = pickle.load(file)

fig_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}"
if not os.path.exists(fig_output_dir):
    os.mkdir(fig_output_dir)

One file yasa detection 

In [67]:
# sub = '32'
# datafile = 2
# id = f"{sub}_{datafile}"

# datafile_data = allsubsdata_perFile[id]['data'] ## shape (electrode, time)
# datafile_scoring = allsubsdata_perFile[id]['scoring'] ## shape (time/sampling/30)

# exmp_scoring_upsampled = yasa.hypno_upsample_to_data(datafile_scoring, 1/30, datafile_data, sf_data=configu['sample_freq'], verbose=True)
# sp = yasa.spindles_detect(datafile_data,sf=configu['sample_freq'], hypno = exmp_scoring_upsampled,include=[2, 3],ch_names=configu['electrodes_names'],multi_only=True)
# print(sp)
# print(sp.summary())
# # sp.plot_average(center='Peak',time_before = 1,time_after = 1)
# # plt.show()
# #%matplotlib widget
# #sp.plot_detection()

In [68]:
# %matplotlib inline
# sp.plot_average(center='Peak', time_before=0.1, time_after=0.1);

Batch yasa detection and save

In [69]:
# yasa_output_dir = f"{configu['outputs_dir_path']}/{output_dir_name}/yasa"
# if not os.path.exists(yasa_output_dir):
#     os.mkdir(yasa_output_dir)

# for id in allsubsdata_perFile:
#     datafile_data = allsubsdata_perFile[id]['data'] ## shape (electrode, time)
#     datafile_scoring = allsubsdata_perFile[id]['scoring'] ## shape (time/sampling/30)
#     if 2 not in datafile_scoring or 3 not in datafile_scoring:
#         continue

#     exmp_scoring_upsampled = yasa.hypno_upsample_to_data(datafile_scoring, 1/30, datafile_data, sf_data=configu['sample_freq'], verbose=False)
#     sp = yasa.spindles_detect(datafile_data,sf=configu['sample_freq'], hypno = exmp_scoring_upsampled,include=[2, 3],ch_names=configu['electrodes_names'],multi_only=True)
#     #summary_df  = sp.summary()
#     summary_df.to_csv(f"{yasa_output_dir}\\{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_yasaSpindles.csv")
#     sp.plot_average(center='Peak', time_before=1, time_after=1)
#     plt.savefig(f"{yasa_output_dir}\\{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_yasaSpindles_average.png")

One file Nir&Andrillon Detection

In [70]:
# import matlab
# import matlab.engine

# sub = '32'
# datafile = 2 
# id = f"{sub}_{datafile}"

# datafile_data = allsubsdata_perFile[id]['data'] ## shape (electrode, time)
# datafile_scoring = allsubsdata_perFile[id]['scoring'] ## shape (time/sampling/30)
# datafile_data_cz = datafile_data[1,:]
# exmp_scoring_upsampled = yasa.hypno_upsample_to_data(datafile_scoring, 1/30, datafile_data_cz, sf_data=configu['sample_freq'], verbose=True)

# eng = matlab.engine.start_matlab()
# eng.SpindlesDetectionAlgorithm_fieldtrip(np.array(datafile_data_cz), configu['sample_freq'], np.array(exmp_scoring_upsampled))

# #spindles, spindlesOutEnergy, spindlesOutDuration, rej_cand = eng.SpindlesDetectionAlgorithm_fieldtrip(datafile_data_cz, configu['sample_freq'], exmp_scoring_upsampled)

# eng.quit()

Batch Nir&Andrillon Detection

In [71]:
configu['electrodes_names']

array(['E127', 'E94', 'E126', 'E1', 'E31', 'E9', 'E22', 'E36', 'E104',
       'E70', 'E83', 'E24', 'E124', 'E52', 'E92', 'Cz'], dtype=object)

In [81]:
import matlab
import matlab.engine
import os

def deleteAuxANFiles(andriNir_aux_output_dir):
    all_files_in_outputDir = os.listdir(andriNir_output_dir)
    for file_in_outputDir in all_files_in_outputDir:
        if file_in_outputDir.endswith(".mat") and not file_in_outputDir.startswith("Spindles_andrillonNir"):
            os.remove(os.path.join( andriNir_output_dir, file_in_outputDir ))

andriNir_code_dir = "C:\\Users\\User\\Cloud-Drive\\AnatArzi\\scripts\\Matlab_scripts - Sharons\\Andrillon_Nir_Algo"
andriNir_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}\\Andrillon_Nir"
andriNir_aux_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}\\Andrillon_Nir\\aux_mats"
if not os.path.exists(andriNir_output_dir):
    os.mkdir(andriNir_output_dir)


electrodes_names_eventDetect = ['Cz']

for id in allsubsdata_perFile:
    datafile_data = allsubsdata_perFile[id]['data'] ## shape (electrode, time)
    datafile_scoring = allsubsdata_perFile[id]['scoring'] ## shape (time/sampling/30)
    scoring_upsampled = yasa.hypno_upsample_to_data(datafile_scoring, 1/30, datafile_data, sf_data=configu['sample_freq'], verbose=True)

    for electrode_name_eventDetect in electrodes_names_eventDetect:
        curr_electrode_num = np.where(configu['electrodes_names'] == electrode_name_eventDetect)[0][0]

        ## create aux files to use in MATLAB
        datafile_1elect_eeg = datafile_data[curr_electrode_num,:]
        if 2 not in datafile_scoring or 3 not in datafile_scoring:
            continue
        
        mat_to_save =  {'datafile_data': datafile_1elect_eeg, 'scoring_upsampled': scoring_upsampled, 'sample_freq': configu['sample_freq'], 'electrode_name':electrode_name_eventDetect}
        scipy.io.savemat(f"{andriNir_aux_output_dir}\\{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_{electrode_name_eventDetect}AndScoring.mat",mat_to_save)

        ## run Andrillon & Nir SS detection over all subjects files
        eng = matlab.engine.start_matlab()
        eng.cd(andriNir_code_dir, nargout=0)
        out = eng.batch_useAndrillonNirEventDetection(andriNir_aux_output_dir, andriNir_output_dir,nargout=0)
        eng.quit()

        ## add the spindles data to the main subject dictionary
        try: 
            spindles_file_name = f"{andriNir_output_dir}\\Spindles_andrillonNir_{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_{electrode_name_eventDetect}AndScoring.mat"
            matlabImport = scipy.io.loadmat(spindles_file_name, simplify_cells=True)
            print(f"imported sub file at: {allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}")
        except Exception: 
            print(f"Error importing spindles sub file at: {allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}")
            continue
        allsubsdata_perFile[id][f'SS_algo-AN@@{electrode_name_eventDetect}'] = matlabImport['spindles'] 
        allsubsdata_perFile[id][f'SW_algo-AN@@{electrode_name_eventDetect}']  = matlabImport['allWaves']
    
deleteAuxANFiles(andriNir_aux_output_dir)
        

27-ינו-23 17:25:46 | WARNING | Hypnogram is SHORTER than data by 4.94 seconds. Padding hypnogram with last value to match data.size.
27-ינו-23 17:25:50 | WARNING | Hypnogram is SHORTER than data by 26.41 seconds. Padding hypnogram with last value to match data.size.


imported sub file at: 32_1


27-ינו-23 17:25:53 | WARNING | Hypnogram is SHORTER than data by 21.62 seconds. Padding hypnogram with last value to match data.size.
27-ינו-23 17:25:53 | WARNING | Hypnogram is SHORTER than data by 25.70 seconds. Padding hypnogram with last value to match data.size.


imported sub file at: 32_2


MatlabExecutionError: 
  File C:\Users\User\Cloud-Drive\AnatArzi\scripts\Matlab_scripts - Sharons\Andrillon_Nir_Algo\batch_useAndrillonNirEventDetection.m, line 16, in batch_useAndrillonNirEventDetection
Unable to find file or directory 'C:\Users\User\Cloud-Drive\BigFiles\OmissionExpOutput\eventDetection\done_on_imported\Andrillon_Nir\33_1_CzAndScoring.mat'.


In [ ]:
def excludeNonN23Events():
    for id in allsubsdata_perFile:
        if 'SS_algo-AN' not in allsubsdata_perFile[id]: continue

        ss_ind_for_deletion = np.asarray([],dtype='int')
        for ind_ss, ss in enumerate(allsubsdata_perFile[id]['SS_algo-AN']):
            if ss[-1] != 2 and ss[-1] != 3:
                ss_ind_for_deletion = np.append(ss_ind_for_deletion, ind_ss)

        if np.size(ss_ind_for_deletion) ==0: continue

        allsubsdata_perFile[id]['SS_algo-AN_n2n3'] = np.delete(allsubsdata_perFile[id]['SS_algo-AN'], ss_ind_for_deletion, axis=0)

    for id in allsubsdata_perFile:
        if 'SW_algo-AN' not in allsubsdata_perFile[id]: continue

        sw_ind_for_deletion = np.asarray([],dtype='int')
        for ind_ss, ss in enumerate(allsubsdata_perFile[id]['SW_algo-AN']):
            if ss[-1] != 2 and ss[-1] != 3:
                sw_ind_for_deletion = np.append(sw_ind_for_deletion, ind_ss)

        if np.size(sw_ind_for_deletion) ==0: continue

        allsubsdata_perFile[id]['SW_algo-AN_n2n3'] = np.delete(allsubsdata_perFile[id]['SW_algo-AN'], sw_ind_for_deletion, axis=0)

#excludeNonN23Events() ## deprecated: names of dic items arent updated!

In [ ]:
# Kcomp_algo-manu@@E22
# SS_algo-AN_n2n3
# SS_algo-manu@@Cz

Change scoring to fit EDF_viewer events format

In [ ]:
def format_scoring_to_edfViewer(score):
    if score == 0:
        return 'W'
    elif score ==1:
        return 'N1'
    elif score ==2:
        return 'N2'
    elif score ==3:
        return 'N3'
    elif score ==4:
        return 'TREM'
    elif score ==5:
        return 'PREM'
    elif score ==6:
        return 'MOVE'
    elif score ==7:
        return 'ARTIFACT'
    else:
        Exception('no such score')
        

for id in allsubsdata_perFile:
    curr_file_scoring = allsubsdata_perFile[id]['scoring']
    new_format_score = np.zeros((len(curr_file_scoring),3), dtype=object)
    for ind, score in enumerate(curr_file_scoring):
            new_format_score[ind,:] = [30*ind,30,format_scoring_to_edfViewer(score)] ## onset (sec), duration, desc

    allsubsdata_perFile[id]['scoring_efdViewFormat'] = new_format_score
    # np.savetxt('test2.txt', new_format_score, delimiter='\t',fmt='%s')

Change spindles events to fit EDF_viewer events format

In [ ]:
# [spindleStartTime spindleEndTime peakTime peakEnergy peakEnergyNorm freqSpindle spindleDuration/SR PowerSP PowerAlpha currentStage];
def EDFViewerformat(key_name):
        curr_file_ss = allsubsdata_perFile[id][key_name]
        new_format_ss = np.zeros((np.shape(curr_file_ss)[0],3), dtype=object)
        for ind, ss in enumerate(new_format_ss):
                SpindleStartTime = curr_file_ss[ind,0] / np.double(configu['sample_freq'])  
                SpindleDuration = (curr_file_ss[ind,1]  -curr_file_ss[ind,0]) / np.double(configu['sample_freq'])  
                new_format_ss[ind,:] = [SpindleStartTime,SpindleDuration,key_name] ## onset (sec), duration,desc
        allsubsdata_perFile[id][f'{key_name}_efdViewFormat'] = new_format_ss

edfViewFormat_events_output_dir = f'{fig_output_dir}\\EDFViewFormat_events'
if not os.path.exists(edfViewFormat_events_output_dir):
    os.mkdir(edfViewFormat_events_output_dir)

events_types_for_save = ['SS_algo-AN','SW_algo-AN']
for event_type in events_types_for_save: 
        for id in allsubsdata_perFile:
                events_type_found = [ v for k, v in allsubsdata_perFile[id].items() if event_type in k]
                if np.size(events_type_found) == 0: 
                        print(f"no {event_type} for sub {id}")
                        continue
                for key in events_type_found: EDFViewerformat(key)

In [ ]:
# Save events (scoring and ss) per file, in a format suitable for EDF_viewer
for id in allsubsdata_perFile:
        filename = f"{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_events"
        all_events_with_header = np.asarray([['Onset',"Duration","Annotation"]],dtype=object)
        for event_type in events_types_for_save:
                events_type_found = [ v for k, v in allsubsdata_perFile[id].items() if (event_type in k) and ('efdViewFormat' in k)]
                if np.size(events_type_found) == 0: continue
                for event_type_found in events_type_found:
                        all_events_with_header = np.concatenate((all_events_with_header,allsubsdata_perFile[id][event_type_found]),dtype=object)
        np.savetxt(f"{edfViewFormat_events_output_dir}\\{filename}.txt", all_events_with_header, delimiter='\t',fmt='%s')
